Name: Tchangmena A Nken Allassan

Creation_date: 15/10/2021

Update_date: 19/10/2021

Here we are going to the train the data from the normal train and test set. GPU needed, to perform training.

In [ ]:
#import necessary libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import json
import pandas as pd
import keras
import cv2
from tqdm import  tqdm
import os

In [ ]:
#unzipping the data files
#!unzip "drive/MyDrive/Train.zip"

In [ ]:
#!unzip "drive/MyDrive/Test.zip"

In [ ]:
train_path='Train/'
test_path='Test/'

#Data Preparation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
BATCH_SIZE=64
IMAGE_SIZE=(224,224,3)

In [ ]:
train_generator=ImageDataGenerator(rescale=1./255)

train_data=train_generator.flow_from_directory(train_path,
                                               target_size=(224,224),
                                               color_mode="rgb",
                                               class_mode="categorical",
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               subset='training')

val_data=train_generator.flow_from_directory(train_path,
                                               target_size=(224,224),
                                               color_mode="rgb",
                                               class_mode="categorical",
                                               batch_size=BATCH_SIZE,
                                               shuffle=False,
                                               subset='validation')

test_generator=ImageDataGenerator(rescale=1./255)
test_data=test_generator.flow_from_directory(test_path,
                                             target_size=(224,224),
                                             color_mode="rgb",
                                             class_mode="categorical",
                                             batch_size=BATCH_SIZE,
                                             shuffle=False,
                                             )

Found 25821 images belonging to 4 classes.
Found 6454 images belonging to 4 classes.
Found 13414 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense,Flatten,Dropout,Input
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
base_model=VGG16(weights="imagenet",include_top=False,input_tensor=Input(shape=IMAGE_SIZE))

head_model=base_model.output
head_model=Flatten(name="flatten")(head_model)
head_model=Dense(4096,activation='relu')(head_model)
head_model=Dropout(0.5)(head_model)
head_model=Dense(4096,activation='relu')(head_model)
head_model=Dropout(0.5)(head_model)
head_model=Dense(4,activation='softmax')(head_model)

for layers in base_model.layers:
  layers.trainable=False

model=Model(inputs=base_model.input,outputs=head_model)
opt=Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
early_stop=EarlyStopping(patience=2,monitor='val_loss')

In [ ]:
result=model.fit_generator(train_data,epochs=20,validation_data=val_data,
                           callbacks=[early_stop])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
404/404 [==============================] - 308s 653ms/step - loss: 0.6075 - accuracy: 0.8725 - val_loss: 0.2022 - val_accuracy: 0.9324
Epoch 2/20
404/404 [==============================] - 242s 599ms/step - loss: 0.3013 - accuracy: 0.8936 - val_loss: 0.2048 - val_accuracy: 0.9394
Epoch 3/20
404/404 [==============================] - 242s 600ms/step - loss: 0.2584 - accuracy: 0.9139 - val_loss: 0.2165 - val_accuracy: 0.9303


In [ ]:
model.save('koubrah_pretrained.h5','drive/MyDrive/')